<a href="https://colab.research.google.com/github/zihadbappy/bnfake/blob/main/Md_Gulzar_Hussain_LSTM_BiLSTM_CNN%2BLSTM_CNN%2BBiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [291]:
!git clone https://github.com/zihadbappy/FakeDatasets.git

fatal: destination path 'FakeDatasets' already exists and is not an empty directory.


In [292]:
import pandas as pd
Dataset= pd.read_excel('/content/FakeDatasets/FinalData1.xlsx')

In [293]:
# Dataset

In [294]:
df= Dataset.filter(['Title','Statement', 'Class'], axis=1)

In [295]:
df.loc[(df.Class=='Real'),'Class']=1
df.loc[(df.Class=='Fake'),'Class']=0

In [296]:
df['Class']=df['Class'].astype('int')
df['Class'].dtypes

dtype('int64')

In [297]:
df.columns=['headline', 'content', 'label']

In [298]:
df.groupby('label').agg('count')

,headline,content
label,,
0,993,993
1,1548,1548


In [299]:
df = df.drop_duplicates(subset='headline', keep="first")


In [300]:
df.groupby('label').agg('count')

,headline,content
label,,
0,948,948
1,1545,1545


In [301]:
df.headline.duplicated().sum()

0

In [302]:
df.groupby('label').agg('count')

,headline,content
label,,
0,948,948
1,1545,1545


In [303]:
import re
from re import sub

In [304]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [305]:
df['headline'] = df.headline.apply(lambda x: preprocessing(str(x)))
df['content'] = df.content.apply(lambda x: preprocessing(str(x)))

In [306]:
df['content'] = df['content'].str.replace(r'\n', ' ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [307]:
df = df.reset_index(drop=True)

In [308]:
pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [309]:
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits

btokenizer = BasicTokenizer()

word_list= []
def clean_text(text):
    tokens = btokenizer.tokenize(text)
    filtered = []
    for i in tokens:
        if i in stopwords:
            continue
    
        if i in punctuations + '‘' + '’':
            continue
    
        filtered.append(i)
        word_list.append(i)
    return " ".join(filtered)

# print("********** Before ***************")
# text = df.iloc[1]['content']
# print(text)
# print("\n********** After ***************")
# print(clean_text(text))

In [310]:
df['content'] = df.content.apply(clean_text)

In [311]:
df.to_csv('MNB-SVC.csv',encoding='utf-8-sig', index=False)

In [312]:
len(set(word_list))

65759

In [313]:
X=df.content
Y=df.label

In [314]:
Y.value_counts()

1    1545
0     948
Name: label, dtype: int64

In [315]:
X.shape

(2493,)

In [316]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping

Onehot Representation


In [317]:
voc_size=65700

In [318]:
onehot_repr=[one_hot(words,voc_size)for words in X]

In [319]:
len(onehot_repr)

2493

In [320]:
news_length=300
embedded_news=pad_sequences(onehot_repr,padding='pre',maxlen=news_length)
print(embedded_news)

[[    0     0     0 ... 26796 45932 30724]
 [    0     0     0 ... 18907 14285 49169]
 [ 8113 31823 57969 ... 28688 20760 28688]
 ...
 [    0     0     0 ... 11978 64573 19323]
 [    0     0     0 ...  1019 30936 62707]
 [    0     0     0 ... 56292 55150 19323]]


In [321]:
embedded_news[0]


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [322]:
len(embedded_news),Y.shape

(2493, (2493,))

In [323]:
import numpy as np
X_final=np.array(embedded_news)
y_final=np.array(Y)

In [324]:
X_final.shape,y_final.shape

((2493, 300), (2493,))

In [325]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=33, stratify=df['label'])

# **LSTM MODEL**

In [326]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 300, 40)           2628000   
                                                                 
 lstm_16 (LSTM)              (None, 100)               56400     
                                                                 
 dense_16 (Dense)            (None, 1)                 101       
                                                                 
Total params: 2,684,501
Trainable params: 2,684,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
28/28 [==============================] - 3s 45ms/step - loss: 0.6217 - accuracy: 0.6195 - val_loss: 0.3997 - val_accuracy: 0.7687
Epoch 2/12
28/28 [==============================] - 1s 23ms/step - loss: 0.1493 - accuracy: 0.9576 - val_loss: 0.0710 - val_accuracy: 0.9733
Epoch 3/12
28/28 [==

# **CNN+LSTM MODEL**

In [327]:
from keras.layers import Dense,Conv1D,MaxPooling1D

In [328]:
embedding_vecor_length = 40
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 300, 40)           2628000   
                                                                 
 conv1d_8 (Conv1D)           (None, 300, 32)           3872      
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 lstm_17 (LSTM)              (None, 100)               53200     
                                                                 
 dense_17 (Dense)            (None, 1)                 101       
                                                                 
Total params: 2,685,173
Trainable params: 2,685,173
Non-trainable params: 0
___________________________________________

LSTM and CNN+LSTM evaluation

In [329]:
y_pred1=model1.predict(X_test)
y_pred2=model2.predict(X_test)

In [330]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [331]:
print("LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred1)))
print("CNN+LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred2)))

LSTM accuracy:0.9786
CNN+LSTM accuracy:0.9853


In [332]:
print("LSTM:\n", classification_report(y_test,np.around(y_pred1),digits=4))
print("\nCNN+LSTM:\n", classification_report(y_test,np.around(y_pred2),digits=4))

LSTM:
               precision    recall  f1-score   support

           0     0.9786    0.9648    0.9716       284
           1     0.9786    0.9871    0.9828       464

    accuracy                         0.9786       748
   macro avg     0.9786    0.9759    0.9772       748
weighted avg     0.9786    0.9786    0.9786       748


CNN+LSTM:
               precision    recall  f1-score   support

           0     0.9892    0.9718    0.9805       284
           1     0.9829    0.9935    0.9882       464

    accuracy                         0.9853       748
   macro avg     0.9861    0.9827    0.9843       748
weighted avg     0.9853    0.9853    0.9853       748



In [333]:
print("LSTM confusion matrix:\n ",confusion_matrix(y_test,np.around(y_pred1)))
print("\nCNN+LSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred2)))

LSTM confusion matrix:
  [[274  10]
 [  6 458]]

CNN+LSTM confusion matrix:
 [[276   8]
 [  3 461]]


#**Bi-LSTM MODEL**

In [334]:
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 300, 40)           2628000   
                                                                 
 bidirectional_8 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 201       
                                                                 
Total params: 2,741,001
Trainable params: 2,741,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/12
28/28 [==============================] - 5s 71ms/step - loss: 0.6432 - accuracy: 0.6287 - val

## **CNN+BiLSTM MODEL**

In [335]:
embedding_vecor_length = 40
model4 = Sequential()
model4.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model4.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Bidirectional(LSTM(100)))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model4.summary())
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 300, 40)           2628000   
                                                                 
 conv1d_9 (Conv1D)           (None, 300, 32)           3872      
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 bidirectional_9 (Bidirectio  (None, 200)              106400    
 nal)                                                            
                                                                 
 dense_19 (Dense)            (None, 1)                 201       
                                                                 
Total params: 2,738,473
Trainable params: 2,738,473
N

In [336]:
y_pred3=model3.predict(X_test)
y_pred4=model4.predict(X_test)

In [337]:
print("BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred3)))
print("CNN+BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred4)))

BiLSTM accuracy:0.9866
CNN+BiLSTM accuracy:0.9706


In [338]:
print("BiLSTM:\n",classification_report(y_test,np.around(y_pred3),digits=4))
print("\nCNN+BiLSTM:\n",classification_report(y_test,np.around(y_pred4),digits=4))

BiLSTM:
               precision    recall  f1-score   support

           0     0.9824    0.9824    0.9824       284
           1     0.9892    0.9892    0.9892       464

    accuracy                         0.9866       748
   macro avg     0.9858    0.9858    0.9858       748
weighted avg     0.9866    0.9866    0.9866       748


CNN+BiLSTM:
               precision    recall  f1-score   support

           0     0.9888    0.9331    0.9601       284
           1     0.9604    0.9935    0.9767       464

    accuracy                         0.9706       748
   macro avg     0.9746    0.9633    0.9684       748
weighted avg     0.9712    0.9706    0.9704       748



In [339]:
print("BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred3)))
print("\nCNN+BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred4)))

BiLSTM confusion matrix:
 [[279   5]
 [  5 459]]

CNN+BiLSTM confusion matrix:
 [[265  19]
 [  3 461]]
